# Tugas Besar IF5250 (Mini Transformer)
Nama: Dzikri Muhammad Raditya Putra

NIM: 18221144

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datasets import Dataset, Features, Sequence, Value
from tokenizers import BertWordPieceTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
from transformers import BertTokenizerFast

In [2]:
import kagglehub

path = kagglehub.dataset_download("debasisdotcom/name-entity-recognition-ner-dataset")

print("Path to dataset files:", path)

100%|██████████| 3.17M/3.17M [00:00<00:00, 138MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/debasisdotcom/name-entity-recognition-ner-dataset/versions/1


In [3]:
import os

for filename in os.listdir(path):
    print(filename)

NER dataset.csv


In [4]:
import pandas as pd

df = pd.read_csv(os.path.join(path, 'NER dataset.csv'), encoding='windows-1252')
print(len(df))

1048575


In [5]:
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')

<ipython-input-5-aea47a665d3e>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sentence #'] = df['Sentence #'].fillna(method='ffill')


In [6]:
df[:10]

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [7]:
df.to_csv("data_text.txt")

In [8]:
grouped = df.groupby("Sentence #").agg(list)
grouped = grouped[:int(len(grouped) * 0.1)]


In [9]:
len(grouped)

4795

In [10]:
sentences = grouped['Word'].tolist()
ner_tags = grouped['Tag'].tolist()

In [11]:
len(ner_tags)

4795

In [12]:
len(sentences)

4795

In [13]:
grouped[:4]

,Word,POS,Tag
Sentence #,,,
Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]"


In [14]:
sentences[:4]

[['Thousands',
  'of',
  'demonstrators',
  'have',
  'marched',
  'through',
  'London',
  'to',
  'protest',
  'the',
  'war',
  'in',
  'Iraq',
  'and',
  'demand',
  'the',
  'withdrawal',
  'of',
  'British',
  'troops',
  'from',
  'that',
  'country',
  '.'],
 ['Iranian',
  'officials',
  'say',
  'they',
  'expect',
  'to',
  'get',
  'access',
  'to',
  'sealed',
  'sensitive',
  'parts',
  'of',
  'the',
  'plant',
  'Wednesday',
  ',',
  'after',
  'an',
  'IAEA',
  'surveillance',
  'system',
  'begins',
  'functioning',
  '.'],
 ['Helicopter',
  'gunships',
  'Saturday',
  'pounded',
  'militant',
  'hideouts',
  'in',
  'the',
  'Orakzai',
  'tribal',
  'region',
  ',',
  'where',
  'many',
  'Taliban',
  'militants',
  'are',
  'believed',
  'to',
  'have',
  'fled',
  'to',
  'avoid',
  'an',
  'earlier',
  'military',
  'offensive',
  'in',
  'nearby',
  'South',
  'Waziristan',
  '.'],
 ['They',
  'left',
  'after',
  'a',
  'tense',
  'hour-long',
  'standoff',
  'wi

In [15]:
ner_tags[:4]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

## Label encoding

In [16]:
label_encoder = LabelEncoder()
flat_tags = [tag for sublist in ner_tags for tag in sublist]
label_encoder.fit(flat_tags)
encoded_tags = [label_encoder.transform(tags).tolist() for tags in ner_tags]
label_list = label_encoder.classes_.tolist()
num_classes = len(label_list)

In [17]:
num_classes

17

In [18]:
label_list

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O']

In [19]:
encoded_tags[:4]

[[16,
  16,
  16,
  16,
  16,
  16,
  2,
  16,
  16,
  16,
  16,
  16,
  2,
  16,
  16,
  16,
  16,
  16,
  3,
  16,
  16,
  16,
  16,
  16],
 [3,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  7,
  16,
  16,
  16,
  5,
  16,
  16,
  16,
  16,
  16],
 [16,
  16,
  7,
  16,
  16,
  16,
  16,
  16,
  2,
  16,
  16,
  16,
  16,
  16,
  5,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  2,
  10,
  16],
 [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]]

# Create Dataset

In [20]:
print(f"tokens: {sentences[0]}")
print(f"tag encoded: {encoded_tags[0]}")

tokens: ['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
tag encoded: [16, 16, 16, 16, 16, 16, 2, 16, 16, 16, 16, 16, 2, 16, 16, 16, 16, 16, 3, 16, 16, 16, 16, 16]


In [21]:
data_dicts = [{"tokens": tokens, "ner_tags": tags} for tokens, tags in zip(sentences, encoded_tags)]

features = Features({
    'tokens': Sequence(Value(dtype='string')),
    'ner_tags': Sequence(Value(dtype='int64'))
})

dataset = Dataset.from_list(data_dicts, features=features)

In [22]:
train_idx = int(len(dataset)*0.8)
train_dataset = dataset.select(range(0, train_idx))
test_dataset = dataset.select(range(train_idx, len(dataset)))

In [23]:
dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 4795
})

In [24]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 3836
})

In [25]:
test_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 959
})

In [26]:
data_dicts[0]

{'tokens': ['Thousands',
  'of',
  'demonstrators',
  'have',
  'marched',
  'through',
  'London',
  'to',
  'protest',
  'the',
  'war',
  'in',
  'Iraq',
  'and',
  'demand',
  'the',
  'withdrawal',
  'of',
  'British',
  'troops',
  'from',
  'that',
  'country',
  '.'],
 'ner_tags': [16,
  16,
  16,
  16,
  16,
  16,
  2,
  16,
  16,
  16,
  16,
  16,
  2,
  16,
  16,
  16,
  16,
  16,
  3,
  16,
  16,
  16,
  16,
  16]}

## Tokenize

In [27]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [28]:
def encode(example):
    tokens = example["tokens"]
    encoding = tokenizer(tokens, is_split_into_words=True, truncation=True, padding="max_length", max_length=128)
    word_ids = encoding.word_ids()
    labels = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(0)
        elif word_idx != previous_word_idx:
            labels.append(example['ner_tags'][word_idx])
        else:
            labels.append(example['ner_tags'][word_idx])
        previous_word_idx = word_idx
    encoding["labels"] = labels
    return encoding


In [29]:
def generate_batch(dataset, tokenizer, batch_size=32):
    encoded = dataset.map(encode)

    def gen():
        for example in encoded:
            input_ids = np.array(example["input_ids"], dtype=np.int64)
            attention_mask = np.array(example["attention_mask"], dtype=np.int64)
            labels = np.array(example["labels"], dtype=np.int64)

            yield (
                {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                },
                labels
            )

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(128,), dtype=tf.int64),
                "attention_mask": tf.TensorSpec(shape=(128,), dtype=tf.int64),
            },
            tf.TensorSpec(shape=(128,), dtype=tf.int64)
        )
    ).repeat().batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [30]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 3836
})

In [31]:
train_batches = generate_batch(train_dataset, tokenizer)
test_batches = generate_batch(test_dataset, tokenizer)

Map:   0%|          | 0/3836 [00:00<?, ? examples/s]

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

In [32]:
train_batches

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 128), dtype=tf.int64, name=None))>

## Positional Embedding

In [33]:
class RelativePositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model, max_len=128):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len
        self.rel_embeddings = self.add_weight(
            shape=(2 * max_len - 1, d_model),
            initializer='random_normal',
            trainable=True,
            name='rel_pos_embedding'
        )

    def call(self, x):
        batch_size, seq_len = tf.shape(x)[0], tf.shape(x)[1]
        pos_indices = tf.range(seq_len)
        rel_pos_matrix = pos_indices[None, :] - pos_indices[:, None] + self.max_len - 1

        rel_pos_matrix = tf.clip_by_value(rel_pos_matrix, 0, 2 * self.max_len - 2)

        rel_pos_embed = tf.gather(self.rel_embeddings, rel_pos_matrix)
        return rel_pos_embed

Digunakan relative positional embedding untuk mendapatkan informasi terkait posisi relatif antar token dalam sequence of token (rel_pos_embed). Pada tahapan ini, dibuat terlebih dahulu relation embedding (rel_embedding) yang berupa matrix yang memetakan posisi relatif ke vector embedding untuk posisi relatif, kemudian dibuat relational position matrix yang merupakan matrix yang berisi posisi relatif antar token. rel_pos_matrix kemudian dilakukan penambahan sehingga nilainya tidak nol (karena akan diperlakukan sebagai index untuk mendapatkan vector embedding untuk posisi relatif). Komponen ini mengeluarkan rel_pos_embed yang berisi matrix vektor relatif position antar token yang mengambil vektor dari rel_embedding berdasarkan rel_pos_matrix. rel_pos_matrix ini selanjutnya akan digunakan dalam mekanisme attention sebagai penambah bias hasil dot product antara vector q dan k dalam mekanisme attention

## Attention

In [34]:
class RelativeGlobalSelfAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.depth = key_dim // num_heads

        self.wq = tf.keras.layers.Dense(key_dim)
        self.wk = tf.keras.layers.Dense(key_dim)
        self.wv = tf.keras.layers.Dense(key_dim)
        self.dense = tf.keras.layers.Dense(key_dim)

        self.dropout = tf.keras.layers.Dropout(dropout)
        self.add = tf.keras.layers.Add()
        self.norm = tf.keras.layers.LayerNormalization()

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, x, mask=None, rel_pos_embed=None):
        batch_size = tf.shape(x)[0]
        seq_len = tf.shape(x)[1]

        q = self.split_heads(self.wq(x), batch_size)
        k = self.split_heads(self.wk(x), batch_size)
        v = self.split_heads(self.wv(x), batch_size)

        matmul_qk = tf.matmul(q, k, transpose_b=True)

        if rel_pos_embed is not None:
            rel_bias = tf.reduce_mean(rel_pos_embed, axis=-1)
            rel_bias = tf.expand_dims(rel_bias, 0)
            rel_bias = tf.expand_dims(rel_bias, 0)
            rel_bias = tf.tile(rel_bias, [batch_size, self.num_heads, 1, 1])

            matmul_qk += rel_bias * 0.1

        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        if mask is not None:
            scaled_attention_logits += mask

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        attention_weights = self.dropout(attention_weights)

        output = tf.matmul(attention_weights, v)
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(output, (batch_size, -1, self.key_dim))

        output = self.dense(concat_attention)
        output = self.add([x, output])
        output = self.norm(output)
        return output

Merupakan bagian yang melakukan mekanisme attention. imput embedded vector untuk suatu token dimasukkan 3 jaringan dense untuk mendapatkan nilai q, v, dan k. Kemudian, dilakukan dot product utuk vector q dan k (k dilakukan transpose dulu agar bisa dilakukan dot product). Kemudian, dicari nilai relative bias dari hasil relative positional embedding yang dilakukan rata-rata untuk setiap vector embedding posisi relatifnya yang kemudian dilakukan perubahan shape dari bias tersebut sehingga memiliki ukuran yang sama dengan hasil dot product q dan k. setelah itu bias tersebut ditambahkan ke hasil dot product q dan k (namun diberi bobot 0.1 agar tidak terlalu dominan). Setelah itu, dilakukan normalisasi dengan membagi akar dari depth. Setelah itu dilakukan softmax dan ditambahkan layer dropout untuk mendapatkan attention weight. Kemudian attention weight tersebut dilakukan dot product dengan nilai v. Kemudian untuk semua attention head hasilnya digabungkan (dengan concat) sehingga didapatkan vector hasil gabungan attention head. Setelah itu hasilnya dimasukkan lagi ke dense layer dan hasilnya ditambahkan dengan input x (bagian residual connection) dan dilakukan normalisasi sebelum dikembalikan menjadi output

## Feedforward

In [35]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.norm(x)
        return x

bagian kode ini berisi feed forward nertwork dengan kumpula dense layer dengan activation relu yag diakhiri dengan layer normalisasi di akhirnya

## Encoder

In [36]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()
        self.attention = RelativeGlobalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)
        self.ffn = FeedForward(d_model, dff, dropout_rate)

    def call(self, x, mask, rel_pos_embed=None):
        x = self.attention(x, mask, rel_pos_embed)
        x = self.ffn(x)
        return x

Bagian kode merupakan layer encoder yang didalamnya terdapat RelativeGlobalSelfAttention dan FeedForward Network

## Transformer

In [37]:
class TransformerEncoder(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, num_classes, dropout_rate=0.1):
        super().__init__()
        self.token_embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.relative_pos_embedding = RelativePositionalEmbedding(d_model)
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs):
        x = inputs['input_ids']
        print(f"inputs: {x}")
        mask = tf.cast(tf.math.equal(x, 0), tf.float32)[:, tf.newaxis, tf.newaxis, :]
        mask = (1.0 - mask) * -1e9

        x = self.token_embedding(x)
        x *= tf.math.sqrt(tf.cast(tf.shape(x)[-1], tf.float32))

        rel_pos_embed = self.relative_pos_embedding(x)

        x = self.dropout(x)
        for enc_layer in self.enc_layers:
            x = enc_layer(x, mask, rel_pos_embed)
        return self.classifier(x)

Bagian ini merupakan keseluruhan arsitektur dari transformer yang didalamnya terdapat embedding layer, relative positinal embedding, encoder layer sebanyak parameter yang ditentukan, dan diakhiri dengan dense layer yang digunakan sebagai classifier

## Model1

In [38]:
model1 = TransformerEncoder(
    num_layers=2,
    d_model=64,
    num_heads=4,
    dff=256,
    vocab_size=len(tokenizer),
    num_classes=num_classes,
    dropout_rate=0.1
)

In [39]:
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['sparse_categorical_accuracy']
)

In [40]:
model1.fit(train_batches, validation_data=test_batches, steps_per_epoch=1000, validation_steps=100, epochs=5)

Epoch 1/5
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2440 - sparse_categorical_accuracy: 0.9458inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 53s 39ms/step - loss: 0.2439 - sparse_categorical_accuracy: 0.9458 - val_loss: 0.0877 - val_sparse_categorical_accuracy: 0.9777
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - loss: 0.0620 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.0902 - val_sparse_categorical_accuracy: 0.9770
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - loss: 0.0554 - sparse_categorical_accuracy: 0.9822 - val_loss: 0.0928 - val_sparse_categorical_accuracy: 0.9764
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 48s 48ms/step - loss: 0.0536 - sparse_categorical_accuracy: 0.9823 - val_loss: 0.0912 - val_sparse_categorical_accuracy: 0.9775


In [41]:
test_loss1, test_accuracy1 = model1.evaluate(test_batches, steps=20, verbose=0)

In [55]:
print(f"test loss model 1: {test_loss1}")
print(f"test accuracy model 2: {test_accuracy1}")

test loss model 1: 0.09138142317533493
test accuracy model 2: 0.9780029058456421


## Model 2

In [42]:
model2 = TransformerEncoder(
    num_layers=4,
    d_model=128,
    num_heads=8,
    dff=512,
    vocab_size=len(tokenizer),
    num_classes=num_classes,
    dropout_rate=0.1
)

In [43]:
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['sparse_categorical_accuracy']
)

In [44]:
model2.fit(train_batches, validation_data=test_batches, steps_per_epoch=1000, validation_steps=100, epochs=5)

Epoch 1/5
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1847 - sparse_categorical_accuracy: 0.9534inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 61s 42ms/step - loss: 0.1846 - sparse_categorical_accuracy: 0.9535 - val_loss: 0.0881 - val_sparse_categorical_accuracy: 0.9772
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - loss: 0.0583 - sparse_categorical_accuracy: 0.9817 - val_loss: 0.0953 - val_sparse_categorical_accuracy: 0.9759
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 38ms/step - loss: 0.0542 - sparse_categorical_accuracy: 0.9822 - val_loss: 0.0972 - val_sparse_categorical_accuracy: 0.9761
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 38ms/step - loss: 0.0527 - sparse_categorical_accuracy: 0.9826 - val_loss: 0.0918 - val_sparse_categorical_accuracy: 0.9771


In [45]:
test_loss2, test_accuracy2 = model2.evaluate(test_batches, steps=20, verbose=0)

In [54]:
print(f"test loss model 2: {test_loss2}")
print(f"test accuracy model 2: {test_accuracy2}")

test loss model 2: 0.10055631399154663
test accuracy model 2: 0.9765259027481079


## Model 3

In [46]:
model3 = TransformerEncoder(
    num_layers=6,
    d_model=256,
    num_heads=16,
    dff=1024,
    vocab_size=len(tokenizer),
    num_classes=num_classes,
    dropout_rate=0.2
)

In [47]:
model3.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['sparse_categorical_accuracy']
)

In [48]:
model3.fit(train_batches, validation_data=test_batches, steps_per_epoch=1000, validation_steps=100, epochs=5)

Epoch 1/5
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.2060 - sparse_categorical_accuracy: 0.9423inputs: Tensor("data:0", shape=(None, 128), dtype=int64)
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 102s 75ms/step - loss: 0.2059 - sparse_categorical_accuracy: 0.9423 - val_loss: 0.0851 - val_sparse_categorical_accuracy: 0.9777
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 74s 74ms/step - loss: 0.0574 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.1000 - val_sparse_categorical_accuracy: 0.9771
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 82ms/step - loss: 0.0529 - sparse_categorical_accuracy: 0.9828 - val_loss: 0.1011 - val_sparse_categorical_accuracy: 0.9775
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 75s 75ms/step - loss: 0.0503 - sparse_categorical_accuracy: 0.9834 - val_loss: 0.1089 - val_sparse_categorical_accuracy: 0.9743

In [49]:
test_loss3, test_accuracy3 = model3.evaluate(test_batches, steps=20, verbose=0)

In [53]:
print(f"test loss model 3: {test_loss3}")
print(f"test accuracy model 3: {test_accuracy3}")

test loss model 3: 0.11703171581029892
test accuracy model 3: 0.975634753704071


## Infefence

In [56]:
def inference(model, tokenizer, label_encoder, sentence, max_length=128):
    if isinstance(sentence, str):
        tokens = sentence.split()
    else:
        tokens = sentence

    encoding = tokenizer(tokens, is_split_into_words=True, truncation=True,
                        padding="max_length", max_length=max_length)

    input_ids = tf.constant([encoding["input_ids"]], dtype=tf.int64)
    attention_mask = tf.constant([encoding["attention_mask"]], dtype=tf.int64)

    inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }

    predictions = model(inputs)
    predicted_labels = tf.argmax(predictions, axis=-1)
    prediction_probs = tf.nn.softmax(predictions, axis=-1)

    pred_labels = predicted_labels.numpy()[0]
    pred_probs = prediction_probs.numpy()[0]

    predicted_tags = []
    confidences = []

    for i, (token, label_id, prob_dist) in enumerate(zip(tokens, pred_labels, pred_probs)):
        if i < len(tokens):
            predicted_tag = label_encoder.inverse_transform([label_id])[0]
            confidence = np.max(prob_dist)
            predicted_tags.append(predicted_tag)
            confidences.append(confidence)

    for token, tag, conf in zip(tokens, predicted_tags, confidences):
        print(f"{token:<15} {tag:<15} {conf:<10.3f}")

    print(f"token: {tokens}")
    print(f"predicted_tags: {predicted_tags}")
    print(f"confidences: {confidences}")
    print(f"raw_prediction: {predictions.numpy()}")
    print(f"prediction_probs: {pred_probs}")

In [57]:
sentence = ["IKN", "is", "the", "capital", "city", "in", "Indonesia"]
inference(model1, tokenizer, label_encoder, sentence)

inputs: [[  101 20912  2078  2003  1996  3007  2103  1999  6239   102     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]
IKN             B-art           1.000     
is              I-per           0.496     
the             I-per           0.331     
capital         O               0.995     
city            O           

In [58]:
sentence = ["Joko", "Widodo", "is", "the", "former", "president", "of", "Indonesia"]
inference(model1, tokenizer, label_encoder, sentence)

inputs: [[  101  8183  3683 15536  3527  3527  2003  1996  2280  2343  1997  6239
    102     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]
Joko            B-art           1.000     
Widodo          B-geo           0.769     
is              I-per           0.985     
the             B-per           0.968     
former          O           